# Backtesting a Mean Reverting Trading Strategy Part 2

## Or How to Retrieve S&P Constituents Historical Data Using Python

## Introduction

## Main

You can find the code below on https://github.com/DinodC/backtesting-mean-reverting-strategy.

Import packages

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from pandas import Series, DataFrame

In [4]:
import pickle

In [5]:
import pandas_datareader.data as web

## S&P Constituents Tickers
In this section, we retrieved the lists pickled from the last article.

Set an id for each index

In [6]:
id = ['sp500', 'sp400', 'sp600']

Create a dictionary to map each id to a tickers file

In [7]:
input_file = {'sp500': 'sp500_barchart.pickle',
              'sp400': 'sp400_barchart.pickle', 
              'sp600': 'sp600_barchart.pickle'} 

Define a dictionary to map each id to a tickers list

In [8]:
sp500_tickers = []
sp400_tickers = []
sp600_tickers = []
sp_tickers = {'sp500': sp500_tickers,
              'sp400': sp400_tickers,
              'sp600': sp600_tickers}

Fill the tickers lists

In [9]:
for i in input_file:
    with open(input_file[i], 'rb') as f:
        
        # Update tickers list        
        sp_tickers[i] = pickle.load(f)

        # Sort tickers list
        sp_tickers[i].sort()
        
    f.close()

## S&P Constituents Historical Data 

Define dictionary of historical data

In [28]:
sp500_data = pd.DataFrame()
sp400_data = pd.DataFrame()
sp600_data = pd.DataFrame()
sp_data = {'sp500': sp500_data,
           'sp400': sp400_data,
           'sp600': sp600_data}

Set the start and date of the historical data

In [29]:
start_date = '2014-01-01'
end_date = '2020-01-01'

Set the source [Investors Exchange (IEX)](https://iextrading.com) to be used

In [30]:
source = 'iex'

Create a dictionary to map each id to an output file

In [31]:
output_file = {'sp500': 'sp500_data.pickle',
               'sp400': 'sp400_data.pickle',
               'sp600': 'sp600_data.pickle'}

Retrieve historical data for each constituent of each S&P index

In [190]:
for i in output_file:
    
    # Retrieve data by groups of 100 tickers or else DataReader gives an error
    
    # S&P 400, 500 and 600 indices     
    data1 = web.DataReader(sp_tickers[i][:100], source, start_date, end_date)
    data2 = web.DataReader(sp_tickers[i][100:200], source, start_date, end_date)
    data3 = web.DataReader(sp_tickers[i][200:300], source, start_date, end_date)
    data4 = web.DataReader(sp_tickers[i][300:400], source, start_date, end_date)
    if i == 'sp500':
        data5 = web.DataReader(sp_tickers[i][400:500], source, start_date, end_date)
        data6 = web.DataReader(sp_tickers[i][500:], source, start_date, end_date)
    elif i == 'sp600':
        data5 = web.DataReader(sp_tickers[i][400:500], source, start_date, end_date)
        data6 = web.DataReader(sp_tickers[i][500:600], source, start_date, end_date)
        data7 = web.DataReader([sp_tickers['sp600'][-1], sp_tickers['sp600'][0]], source, start_date, end_date)
    else:
        pass
    
    # Combine the groups of data
    if i == 'sp400':
        sp_data[i] = pd.concat([data1, data2, data3, data4], axis=1, sort=True)
    elif i == 'sp500':
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6], axis=1, sort=True)
    elif i == 'sp600':
        #
        data6 = pd.concat([data6, data7], axis=1, sort=True)
        data6.drop(columns=sp_tickers['sp600'][0], level=1, inplace=True)
        # 
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6], axis=1, sort=True)
    else:
        pass
        
    # Convert index to datetime
    sp_data[i].index = pd.to_datetime(sp_data[i].index)
    
    # Save S&P constituents historical data to file
    output = open(output_file[i], 'wb')
    pickle.dump(sp_data[i], output)
    output.close()

## S&P 500 Index

Eyeball

In [74]:
sp_data['sp500'].close.shape

(1258, 505)

In [70]:
sp_data['sp500'].close.head()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2014-05-07,37.4775,35.4569,118.4361,77.7918,43.8799,59.7061,21.469,34.7855,70.6963,58.63,...,26.6898,40.9714,85.4843,44.3468,26.7296,34.7561,49.5481,95.9619,27.1487,29.5528
2014-05-08,37.6614,36.4381,121.4206,77.2831,43.3480,59.5766,20.870,34.8215,71.2567,59.08,...,26.3102,41.2198,84.8294,44.3080,26.9330,34.8308,49.6593,95.3575,27.3469,29.3889
2014-05-09,37.7227,36.4477,123.5028,76.9614,43.4394,60.6682,21.590,35.0552,71.1754,59.59,...,25.9390,41.3795,85.0965,44.7737,26.7522,34.7281,49.9143,95.9139,27.4130,29.4467
2014-05-12,38.4174,37.5241,124.1869,77.9193,43.5308,61.6766,21.920,35.3070,71.5821,60.70,...,25.7028,42.1741,85.3302,45.0550,27.1364,35.5124,49.7835,96.7773,27.9511,29.7071
2014-05-13,38.7034,37.4479,122.3625,78.0415,43.3895,61.8431,21.550,35.6576,72.0251,60.81,...,25.7872,41.2278,85.4387,45.1908,27.3171,35.4471,50.0059,97.1610,27.5924,29.6299


In [71]:
sp_data['sp500'].close.tail()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2019-04-30,78.50,34.0816,166.32,200.67,79.39,74.76,277.41,79.56,182.67,289.25,...,56.50,120.14,80.28,51.13,33.36,83.40,104.39,123.16,49.33,101.84
2019-05-01,77.47,33.8124,163.83,210.52,78.89,73.39,277.07,78.74,181.31,283.35,...,55.90,116.64,78.67,50.31,33.26,83.29,101.94,121.22,48.58,101.79
2019-05-02,78.20,33.8523,166.78,209.15,78.47,77.06,264.77,78.78,179.39,279.64,...,55.92,118.90,77.29,51.50,32.95,79.54,101.74,123.21,49.48,103.15
2019-05-03,79.29,34.6899,163.27,211.75,78.71,79.14,271.75,78.69,176.98,285.58,...,56.58,119.02,77.47,55.05,32.95,82.29,102.72,124.31,50.05,103.75
2019-05-06,79.35,34.6500,161.99,208.48,79.26,78.31,268.95,79.07,176.26,283.66,...,56.51,118.81,77.13,54.73,32.65,80.08,102.41,125.55,49.68,103.33


In [72]:
sp_data['sp500'].close.describe()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,51.404909,41.347022,144.761517,133.595312,66.009518,84.710403,158.144399,48.325983,116.473141,138.198267,...,38.739282,57.769132,76.481116,53.000040,26.786743,50.672424,65.875394,111.340300,36.594089,58.390266
std,13.742986,6.213187,24.263837,38.291399,17.897667,9.893054,117.282460,12.173258,30.690718,68.729220,...,8.109465,21.355433,4.895579,7.899517,3.131990,16.164368,15.410218,10.045719,10.817077,19.684172
min,32.258600,24.539800,79.168700,76.961400,42.066600,59.576600,20.870000,33.935700,68.852300,58.630000,...,25.247700,32.609200,59.643400,34.178400,18.532600,28.968100,44.366900,89.236100,19.008100,29.196000
25%,39.210025,36.834425,130.426500,103.196200,52.465825,77.583825,75.497500,39.075500,90.520825,80.530000,...,31.158200,41.398275,73.431000,46.920600,24.215550,35.092625,52.708225,102.267175,26.912550,44.281450
50%,45.612000,40.848200,149.156550,118.565600,57.999400,84.243350,116.905000,42.798100,111.533450,105.250000,...,38.584850,50.991100,76.894250,53.859750,26.573400,47.783300,61.579200,112.391550,31.055250,50.303100
75%,65.146025,46.119900,161.834300,165.866775,82.894300,90.312650,235.045000,57.441150,147.632325,197.072500,...,45.132050,67.980625,79.647950,59.650550,29.469225,66.731600,79.054800,118.848400,46.561225,76.161200
max,81.940000,57.586600,199.159900,230.275400,116.445400,108.207500,449.750000,79.733700,182.670000,289.250000,...,57.360000,139.720000,87.124800,67.795300,35.000000,83.820000,104.390000,130.912800,57.511000,103.750000


## S&P MidCap 400 Index

Eyeball

In [75]:
sp_data['sp400'].close.shape

(1273, 400)

In [76]:
sp_data['sp400'].close.head()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2014-05-07,30.9958,32.2135,42.60,13.4550,31.56,NaN,9.3311,48.9611,52.6823,43.1130,...,22.4894,22.10,43.3536,15.9080,28.2884,24.1291,24.37,407.3629,52.74,74.72
2014-05-08,31.2321,32.1725,41.71,13.2125,31.85,NaN,9.5274,48.9195,52.6251,42.6417,...,22.2154,21.81,43.0260,15.4530,28.1817,24.2915,23.24,406.3601,53.29,74.51
2014-05-09,31.6652,31.9756,42.21,13.4225,32.07,NaN,9.8177,48.8031,52.6060,42.2125,...,22.0474,21.94,43.5655,15.3995,28.3556,23.8807,23.75,407.6382,54.22,74.03
2014-05-12,32.7774,31.9510,43.03,13.9825,32.60,NaN,9.9030,49.1525,52.5488,42.1031,...,22.1535,22.06,44.9818,16.2560,28.7706,24.7804,24.59,412.2492,56.60,74.23
2014-05-13,32.2164,31.6063,43.06,13.8000,32.02,NaN,10.0738,49.0277,52.5870,41.9076,...,22.0474,21.88,44.5868,16.4166,28.7152,24.8187,23.79,412.9473,55.53,73.41


In [77]:
sp_data['sp400'].close.tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2019-04-30,55.69,47.20,32.02,35.52,33.90,23.10,23.78,103.53,70.78,81.45,...,39.06,20.42,108.90,83.85,43.56,15.60,68.08,656.88,40.06,211.14
2019-05-01,54.63,47.11,32.37,34.85,33.47,23.02,23.82,103.06,70.06,80.40,...,38.61,19.89,108.17,82.63,44.48,15.27,67.13,656.90,39.64,205.12
2019-05-02,56.94,47.15,32.26,34.98,33.11,23.90,24.04,102.49,74.93,80.30,...,38.39,20.31,108.52,85.60,43.71,14.39,64.10,655.30,40.19,205.94
2019-05-03,58.75,47.31,33.42,35.42,33.62,24.89,24.12,103.51,74.82,82.05,...,38.09,22.96,112.34,85.80,44.63,16.88,64.45,662.82,40.98,206.46
2019-05-06,59.21,47.11,33.82,35.44,33.68,24.35,23.52,104.22,74.15,82.99,...,37.80,23.14,111.50,87.16,44.58,16.63,65.48,666.31,40.73,206.36


In [78]:
sp_data['sp400'].close.describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,631.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,34.909327,39.195374,49.014141,22.303651,32.394293,52.639473,15.602632,76.950255,55.505354,59.154164,...,29.474494,29.818581,62.577214,30.515298,36.758073,24.782118,53.491701,533.900374,40.569579,104.180636
std,9.430847,4.989257,13.746324,4.036917,3.178308,20.087878,4.050000,21.221802,9.343745,13.537498,...,4.836215,23.447252,14.918453,24.996117,7.052731,8.706155,25.877595,71.303250,13.393709,41.134424
min,20.118600,28.081400,24.750000,13.212500,23.150000,12.570000,8.872200,46.728500,40.309000,38.117400,...,20.741700,3.780000,38.963800,9.233700,25.207900,6.475600,19.560000,400.952700,15.230000,46.930000
25%,25.957675,34.664375,38.800000,19.382500,30.370000,40.975950,12.761875,58.690275,47.042025,45.516725,...,24.323550,12.060000,48.476325,15.568325,30.973750,19.111850,33.045000,471.573150,32.002500,74.200000
50%,34.342100,39.873150,46.660000,22.045000,32.410000,58.780900,14.581400,67.872900,53.269050,58.249600,...,29.680650,21.115000,61.692400,19.160050,34.596250,23.932100,45.830000,535.675250,39.565000,90.600000
75%,42.029000,43.659200,59.717500,24.037500,34.537500,66.666500,17.824475,97.213375,63.204100,73.264800,...,33.317750,44.240000,74.364175,34.149700,42.879800,32.300300,64.747500,598.617225,45.907500,122.737500
max,59.210000,48.853300,82.970000,35.520000,40.130000,84.046800,28.421700,114.972900,74.930000,83.830000,...,39.060000,103.090000,112.340000,99.250000,54.884400,45.690400,114.540000,666.310000,84.960000,235.440000


## S&P SmallCap 600 Index

Eyeball

In [194]:
sp_data['sp600'].close.shape

(1258, 601)

In [195]:
sp_data['sp600'].close.head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2014-05-07,20.00,19.6669,30.3094,36.32,13.13,19.3728,61.22,24.3261,NaN,6.44,...,NaN,20.0707,77.40,9.2157,50.5987,24.5170,NaN,17.4820,23.3058,24.82
2014-05-08,19.00,19.3403,30.3973,35.78,13.14,19.3536,61.36,23.9463,NaN,6.16,...,NaN,19.8117,77.73,9.1766,50.1827,24.8283,NaN,17.8549,23.4036,27.54
2014-05-09,17.57,19.6925,30.3797,35.88,13.26,19.7560,62.85,24.2267,NaN,6.48,...,NaN,19.9735,77.30,9.2092,51.2889,24.7151,NaN,18.3379,23.3449,28.28
2014-05-12,17.44,20.4546,30.1511,36.36,14.06,20.1009,64.66,24.8687,NaN,6.76,...,NaN,20.0868,79.30,9.2678,52.4801,25.1019,NaN,18.5836,24.3523,28.93
2014-05-13,17.26,19.8847,30.1248,37.40,13.64,19.9764,63.44,24.5703,NaN,6.56,...,NaN,19.8683,79.31,9.1571,52.0263,24.7246,NaN,18.5243,23.6579,28.52


In [196]:
sp_data['sp600'].close.tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2019-04-30,12.52,50.21,46.19,48.29,63.50,36.46,80.18,37.97,31.13,21.29,...,4.45,28.24,129.99,12.735,85.59,36.81,21.65,24.85,16.22,26.63
2019-05-01,12.27,49.87,46.22,47.59,63.79,36.12,79.18,36.63,31.02,21.12,...,4.42,28.13,126.81,12.540,84.74,36.56,21.55,24.62,16.01,25.45
2019-05-02,12.09,49.52,46.12,45.91,64.26,36.75,79.07,37.42,30.29,21.27,...,4.39,27.89,129.49,12.500,84.36,37.04,22.31,24.88,16.14,25.79
2019-05-03,12.60,51.81,46.22,46.85,65.30,37.58,80.45,38.36,37.43,21.96,...,4.77,28.36,133.86,12.730,86.49,37.50,22.99,25.47,17.61,26.94
2019-05-06,12.69,49.35,46.18,44.95,65.84,37.39,80.58,38.41,37.62,22.08,...,4.86,28.36,132.62,12.780,87.51,36.98,23.00,25.08,17.02,26.53


In [197]:
sp_data['sp600'].close.describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,128.000000,1258.000000,...,1252.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1070.000000,1258.000000,1258.000000,1258.000000
mean,26.710111,29.180265,37.085760,49.376812,30.567158,36.157897,67.202464,32.501787,29.561609,15.837186,...,7.595914,26.056974,74.416459,10.770695,63.015704,26.620118,17.369857,26.411636,18.892807,22.917675
std,16.985480,7.515236,3.534006,10.213914,15.308510,10.748189,10.021847,5.658112,2.777749,6.638298,...,1.935929,3.247319,26.969176,1.388868,10.428099,5.737581,3.186658,6.750282,4.452323,6.830982
min,8.380000,16.343700,29.352300,31.400000,10.500000,18.903300,45.070000,22.347500,20.930200,6.160000,...,4.390000,19.787400,26.700000,7.392600,43.328400,14.741000,10.508700,12.051000,8.168700,11.450000
25%,15.092500,22.086250,35.148675,40.865000,21.932500,25.625925,59.167500,27.855950,28.581825,10.490000,...,6.082750,22.931625,50.137500,9.810400,54.051150,23.000950,14.878375,21.046925,16.553550,17.500000
50%,20.035000,29.201400,37.413700,49.150000,24.990000,34.559000,67.165000,31.078600,29.701800,13.645000,...,7.020100,26.374200,76.692500,10.814650,60.822250,26.377200,17.724250,26.908300,19.281900,22.075000
75%,33.927500,35.221075,38.989300,56.542500,33.700000,45.666850,73.300000,37.368150,31.029900,20.950000,...,8.849000,29.164850,97.300000,11.701950,71.616575,30.340650,19.498925,30.757175,22.279500,27.045000
max,99.610000,51.810000,46.770000,74.000000,74.890000,57.403000,95.540000,43.220900,37.620000,36.625000,...,12.678000,31.602900,133.860000,14.214700,87.510000,39.469500,24.438200,42.148900,30.787100,41.400000


## Conclusion